In [5]:
from OptiHPLCHandler import EmpowerHandler

In [6]:
# get api address from .env file
import os
from dotenv import load_dotenv

load_dotenv("../.env_vars")
EMPOWER_API_ADDRESS = os.getenv("EMPOWER_API_ADDRESS_PRD")

In [3]:
# import sys
# sys.path.insert(0, "../")

In [7]:
# Create an instance of the EmpowerHandler class
handler = EmpowerHandler(project="WebAPI_test", address=EMPOWER_API_ADDRESS)

handler.connection.default_get_timeout = 300
handler.connection.default_post_timeout = 300

# Get the list of methods, select one, and get the method details
with handler:
    method_list = handler.GetMethodList()  # Get the list of instrument methods
    method_name = "20240430_slcb"
    full_method = handler.GetInstrumentMethod(method_name)

In [ ]:
from applications.generate_basic_robustness_methods import (
    generate_basic_robustness_sampleset,
    generate_basic_robustness_instrument_methods,
)

dict_methods = generate_basic_robustness_instrument_methods()

generate_basic_robustness_sampleset(dict_methods)

In [9]:
from OptiHPLCHandler.utils.validate_gradient_table import validate_gradient_table

gradient_table = [
    {
        "Time": "Initial",
    },
    {
        "Time": "10.0",
    },
    {
        "Time": "9.1",
    },
]

validate_gradient_table(gradient_table)

ValueError: The time in the gradient table row is less than the previousrow. The row is 9.1 and the previous rowis 10.0.